___

<a href='https://www.learntocodeonline.com/'> <img src='files/IMGs/learn to code online.png' /></a>
___

# Plan Our Profiles API

Now that we have created a simple API, we can move on to a profiles API.

Basic requirements:
1. create a new profile (handle new registrations in the system)
    - validate profile data
2. list existing profiles
    - search for profiles via email or name
3. view specific profile (using an ID)
4. update "my profile" of a logged in user
    - update name/email address
    - change password
5. delete profile

URLs for our API:
1. list of all profiles:  **`/api/profile/`**
    - GET (list profiles)
    - POST (create profile)
    <br><br>
2. manage a specific profile:  **`/api/profile/<profile_id>/`**
    - GET (view specific profile)
    - PUT/PATCH (update profile)
    - DELETE (remove profile)

# Create User Profile Serializer

If you would like more information, check out the [ModelSerializer official documentation](http://www.django-rest-framework.org/api-guide/serializers/#modelserializer).

Ensure that you have the **profiles-rest-api** project open in your Atom!

Locate the `serializers.py` file in the profiles_api folder.

Now we will create a ModelSerializer - similar to the HelloSerializer created earlier, but these are designed to be used with models. There is a bunch of extra functionality that makes it really easy to work with existing Django database models.

1. import the following:

    `from . import models` ----- or -----> `from profiles_api import models`


2. Create a new class below your HelloSerializer that will inherit from serializers.ModelSerializer

In [ ]:
class UserProfileSerializer(serializers.ModelSerializer):
    """A serializer for our user profile objects."""
    
    # create a meta class that tells Django REST framework
    # what fields we want to take from our model
    class Meta:
        # assign model we want it to point to (our user profile model)
        model = models.UserProfile
        
        # what fields to use in the serializer
        fields = ('id', 'email', 'name', 'password')
        
        # define extra keyword arguments for your model
        # allows you to tell Django REST framework special attributes
        # you want to apply to those fields above
        # keys in teh dictionary are the fields you want to add custom args to
        extra_kwargs = {
            'password': {
                'write_only': True,
                'style': {'input_type': 'password'}  # custom style for browsable API
            }
        }
        
    # create a special function that overwrites the CREATE functionality
    # this will allow us control over how users are created
    # we want to encrypt the PW as a hash
    def create(self, validated_data):
        """Create and return a new user."""
        
        user = models.UserProfile.objects.create_user(
            email=validated_data['email'],
            name=validated_data['name'],
            password=validated_data['password']
        )
        
        # user.set_password(validated_data['password'])
        # user.save()
        
        return user
    
    # added to avoid clear text issue
    # https://github.com/LondonAppDev/profiles-rest-api/blob/master/profiles_api/serializers.py#L34
    def update(self, instance, validated_data):
        """Handle updating user account"""
        if 'password' in validated_data:
            password = validated_data.pop('password')
            instance.set_password(password)  # saves as hash
 
        return super().update(instance, validated_data)

The **update()** function was added as per [here](https://www.udemy.com/course/django-python/learn/lecture/18186246#questions/5287482).

# [Create User Profiles Viewset](https://www.udemy.com/course/django-python/learn/lecture/6955052#questions/5287482)

We will now create a ViewSet to access the serializer through an endpoint.

Open the `views.py` file in the profiles_app app folder.

Import your model file:  `from . import models` or `from profiles_api import models`

For our user profile API, we will be using a model ViewSet - which is similar a standard ViewSet, except it is specifically designed for managing models through our API. Therefore, it has a lot of the functionality we need already built into it.

The way you use a Model ViewSet is to connect it to a serializer class (like a regular ViewSet) then provide a query set to the Model ViewSet so it knows which objects in the database are going to be managed through the ViewSet.

Scroll to the bottom of the file to add the UserProfileViewSet class.

In [ ]:
class UserProfileViewSet(viewsets.ModelViewSet):
    """Handles creating, reading and updating profiles."""
    # ModelViewSet is a special ViewSet by the Django REST framework
    # that takes care of all of the logic for CRU our model items
    
    # define the serializer class - this serializer has the model
    # in the metadata so it knows which model to look for
    # to handle user objects
    serializer_class = serializers.UserProfileSerializer
    
    # query the set which tells the ViewSet how to retrieve
    # the object from our DB
    queryset = models.UserProfile.objects.all() # this retrieves all of them

The Django REST framework knows the standard functions you would want to perform on a model ViewSet:
- CREATE for new items
- LIST to list models in the database
- UPDATE for partial update
- DESTROY for specific models in the database

# [Register Profile ViewSet With URL Router](https://www.udemy.com/course/django-python/learn/lecture/6955058#questions/5287482)

Open the `urls.py` file in the profiles_api app folder.

Registering is simple - just need to register with a router and Django REST framework will do the rest.

Below the latest router registration add:

In [ ]:
router.register('profile', views.UserProfileViewSet)

**NOTE:** 
When specifying a Model ViewSet, you do not need to provide a base_name... As long as you have already provided the **queryset** in your **views.py** file for **UserProfileViewSet**. Django can figure out the name based on the model assigned to it. Only need to provide if **queryset** not provided or you wish to override the name of the query set that is associated to it.

# [Test Creating A Profile](https://www.udemy.com/course/django-python/learn/lecture/6955062#questions/5287482)

Ensure the vagrant server is running - may have to restart.

1. In git bash, change directory to where you profiles-rest-api project is located:  `cd workspace/profiles-rest-api`
<br><br>
2. `vagrant up`
<br><br>
3. `vagrant ssh`
<br><br>
4. `workon profiles_api`
<br><br>
5. `cd /vagrant/src/profiles_project`
<br><br>
6. Start development server:  `python manage.py runserver 0.0.0.0:8000`

Once you have the server up, head over to the following in Chrome:
**127.0.0.1:8000/api**

Notice on the root API page that we have an additional entry to the list of APIs that are available.

<img src='files\IMGs\views\ProfilesAPI-01.png'>

Click on the link to head over to the **API/profile** of the API.

<img src='files\IMGs\views\ProfilesAPI-02.png'>

This is the **list** or HTTP GET to our **api/profile** endpoint. It lists all users in the system.

## Test POST Without Data

<img src='files\IMGs\views\ProfilesAPI-03.png'>

## Create A New User

<img src='files\IMGs\views\ProfilesAPI-04.png'>

Since the password was set to write_only, it never gets returned by the API.

## Test User Update

Go back to the root of the API for profiles:  **127.0.0.1:8080/api/profile**

<img src='files\IMGs\views\ProfilesAPI-05.png'>

You can now see the different users in the system.

Add the ID to the end of the URL to see the information from the new user you just created:  **127.0.0.1:8080/api/profile/2**

<img src='files\IMGs\views\ProfilesAPI-06.png'>

You can see at the bottom it has given you the option to modify the user profile.

- HTML form allows you to use PUT
<img src='files\IMGs\views\ProfilesAPI-07.png'>

- Raw Data form allows you to use PATCH
<img src='files\IMGs\views\ProfilesAPI-08.png'>

If using RAW DATA, delete the ID (you can't change that) and any other field you don't wish to update. Make the changes you wish to do, then click PATCH.

<img src='files\IMGs\views\ProfilesAPI-09.png'>

You will see that it updated on that screen ...

<img src='files\IMGs\views\ProfilesAPI-10.png'>

As well as at the root of the API:  **127.0.0.1:8080/api/profile**

<img src='files\IMGs\views\ProfilesAPI-11.png'>

## Commit To Git

In your **git bash** program ...

1. go to project directory:  `cd workspace/PROJECTNAME` (in this example **profiles-rest-api**)
2. Call `git add .`
3. Call `git commit -am "Added profile listing and creating feature."`

# Create Permission Class

If you would like more information, check out the [Permissions official documentation](http://www.django-rest-framework.org/api-guide/permissions).

At this point, there is no security applied to your API. Thus anyone can use the API to update any profile. This will be resolved by creating a user profile permissions class and applying to the UserProfile ViewSet.

1. Create a new file in your **profiles_api** application folder:  `permissions.py`

This will be where we store all of our permission classes for our application. The permissions class is a class that Django REST framework uses to determine if the user has the permission to make the changes they're asking.

Each time a request is made to our API, the Django permissions classes are checked to ensure that the request being made is a legal request.

You create permission classes by inheriting from the base permissions class that comes with the Django REST framework. This *permissions* module contains all of the base permission classes in the Django REST framework.

In [ ]:
# import the base permissions module from Django REST framework
from rest_framework import permissions


class UpdateOwnProfile(permissions.BasePermission):
    """Allow users to edit their own profile."""
    
    # this is called every time we make a request to our API
    # the result determines whether the user has the permission
    # to make the request / perform the action
    def has_object_permission(self, request, view, obj):
        """Check user is trying to edit their own profile."""
        
        # allow users to be able to view any profile in the system
        # don't want to apply any permissions if user is simply
        # trying to view a profile
        # This is completed by checking the safe methods list
        # provided by the REST Django framework
        
        # HTTP safe methods are non-destructive - allows you to 
        # retrieve data but not change/modify/delete any objects        
        if request.method in permissions.SAFE_METHODS:
            return True
        
        # making change on own profile?
        return obj.id == request.user.id

# Add Authentication & Permissions To ViewSet

Now that we have created our permissions class, it is time to add it to the ViewSet - or configure it to use the permission. This will be done in the `views.py` file in the **profiles_api** app folder.

1. Need to import the new file first.

    `from . import permissions` or `from profiles_api import permissions`
    
    Or you can simply add `, permissions` to the end of your import line witht he other imports from the same module.


2. Import the token authentication class that comes with the REST framework.

    `from rest_framework.authentication import TokenAuthentication`

    There are multiple ways to authenticate using the REST framework. As stated, we will be using token authentication. It is the most effective & popular way to authenticate with an API.

    Token authnetication works by giving the user a temporary token that it inserts in the headers of the HTTP requests from the Django REST framework. It then uses this token to check that the user has authenticated with the system. It is in effect a password to ensure that changes are made appropriately without forcing the user to constantly put in their password.
    

3. At the bottom of the file, update your **UserProfileViewSet** to use your new permissions class and token authentication.

    The reason these are created as tuples is because you can add multiple authentication classes or permissions classes to a particualr viewset. _(e.g.:  you may also want session authentication that requires a cookie to hold the session variable)_

In [ ]:
...
    # add an authentication class variable below the query set variable
    # ensure a comma is after it to ensure Python knows it's a tuple
    authentication_classes = (TokenAuthentication,)
    
    # define permission classes that will be applied
    permission_classes = (permissions.UpdateUserProfile,)

Although the **TokenAuthenticaion** is what we will be using, you can configure 1 or more type at at time.  This is done by adding all of the classes to your **authentication_classes** class variable.

# Test New Permissions

Once you've saved the files, ensure you are currently connected to the server through gitbash. In the Chrome window, either reload or go to the profile url:  **http://127.0.0.1:8080/api/profile/**

It will still look the same, since viewing profiles & creating new profiles is allowed & part of the HTTP safe methods list. But ... *what happens if you try and modify another user's profile?*

1. Click on the URL and add one of the IDs to the end.

<img src="files/IMGs/permissions-01.png">

You can still see the profile as shown above, however you don't have the option to change or delete the object. Therefore you can see the permissions class is working properly.

## Commit To Git

In your **git bash** program ...

1. go to project directory:  `cd workspace/PROJECTNAME` (in this example **profiles-rest-api**)
2. Call `git add .`
3. Call `git commit -am "Added permissions and authentication to our UserProfileViewSet."`

# Add Search Profiles Feature

This one of the more advanced features to add to your user profile ViewSet. This will provide the avility to search profiles by name or email address. The code will be applied to the **profiles_api** app folder in the `views.py` file.

1. A few more imports will be needed:

`# REST framework filters module to provide filtering capability
from rest_framework import filters`

2. Scroll to bottom of file to add more class variables to your UserProfileViewSet to tell it which filters to use.

In [ ]:
...
    # filters to use
    filter_backends = (filters.SearchFilter,)
    
    #fields to filter by
    search_fields = ('name', 'email')

# Test Searching Profiles

1. After saving your changes, be sure your vagrant server is up and go to your browser.
<br><br>
2. Click on the root your **USer Profile List** on your API page.

<img src="files/IMGs/permissions-02.png">

This shows a list of the user profiles that are in the system.

3. Create a new user to test the search capability.

<img src="files/IMGs/permissions-03.png">

4. Use the filter capability to search for the new user. Go back to the **USer Profile List** - you will notice that there is a new option for filters!

<img src="files/IMGs/permissions-04.png">

When you click on the button, a search field will pop up.

<img src="files/IMGs/permissions-05.png">

5. Search for the name of one of your users.

<img src="files/IMGs/permissions-06.png">

You will also note that you can see what the HTTP call used.

<img src="files/IMGs/permissions-07.png">

It will also be in the URL:  **http://127.0.0.1:8080/api/profile/?search=New**

When using the API from a client application (like a mobile app or website) this is how you would use the search feature. - There won't be a webpage, so it needs to add the filter parameters in the URL.

Also known as **get parameters**. In this instance, we have a *search* **GET parameter** using phrase _New_ for the field. And it will return any user with that name in the username or email.

## Commit To Git

In your **git bash** program ...

1. go to project directory:  `cd workspace/PROJECTNAME` (in this example **profiles-rest-api**)
2. Call `git add .`
3. Call `git commit -am "Added search function to user profiles API."`